In [8]:
# scheduled_oltp_update.py
import pandas as pd
import os
from datetime import datetime
import mysql.connector
import logging

# Set up logging for better visibility
logging.basicConfig(level=logging.INFO)

def extract_data(input_csv):
    """
    Extracts data from a specified CSV file into a pandas DataFrame.
    """
    print(f"📥 Extracting data from '{input_csv}'...")
    try:
        df = pd.read_csv(input_csv, sep=";")
        return df
    except FileNotFoundError:
        print(f"❌ Error: The file '{input_csv}' was not found.")
        return None

def transform_data(df, start_offset_date, end_offset_date):
    """
    Transforms the raw DataFrame by:
    - Filtering by date range
    - Formatting dates & times
    - Mapping store locations
    - Converting currency
    - Regenerating unique transaction IDs
    """
    print("🔄 Transforming data...")

    # Convert date and time
    df['transaction_date'] = pd.to_datetime(df['transaction_date'], errors='coerce')
    df['transaction_time'] = pd.to_datetime(
        df['transaction_time'], format='%H:%M:%S', errors='coerce'
    ).dt.strftime('%H:%M:%S')

    # Filter by date range
    start_date = pd.to_datetime(start_offset_date)
    end_date = pd.to_datetime(end_offset_date)
    df_filtered = df[
        (df['transaction_date'] >= start_date) & (df['transaction_date'] <= end_date)
    ].copy()

    if df_filtered.empty:
        print(f"⚠ No data found for the range {start_offset_date} to {end_offset_date}.")
        return df_filtered

    # Format date for output
    df_filtered['transaction_date'] = df_filtered['transaction_date'].dt.strftime('%Y-%m-%d')

    # Map store locations
    location_map = {
        "Hell's Kitchen": "East Campus",
        "Astoria": "West Campus",
        "Lower Manhattan": "Main Campus"
    }
    df_filtered['store_location'] = df_filtered['store_location'].replace(location_map)

    # Convert unit_price
    df_filtered['unit_price'] = (df_filtered['unit_price'] * 15).round(2)

    # Convert key columns to string safely
    cols_to_convert = ['transaction_id', 'store_id', 'product_id']
    df_filtered[cols_to_convert] = df_filtered[cols_to_convert].apply(
        lambda col: col.dropna().astype(int).astype(str)
    )

    # Force unique transaction IDs by appending the date
    df_filtered['transaction_id'] = (
        df_filtered['transaction_id'].astype(str) + "_" +
        pd.to_datetime(df_filtered['transaction_date']).dt.strftime('%Y%m%d')
    )
    
    # Reorder columns to match the MySQL table structure
    df_transformed = df_filtered.reindex(columns=[
        'transaction_id', 'transaction_date', 'transaction_time', 'transaction_qty',
        'store_id', 'store_location', 'product_id', 'unit_price',
        'product_category', 'product_type', 'product_detail'
    ], fill_value=None)

    return df_transformed


def load_data_to_mysql(df, db_config, table_name):
    """
    Loads transformed data into a MySQL table.
    """
    if df.empty:
        print("⚠ No data to load to MySQL.")
        return

    print("🚀 Loading data to MySQL...")
    try:
        conn = mysql.connector.connect(**db_config)
        cursor = conn.cursor()
        print("Successfully connected to MySQL database.")

        # Prepare the list of tuples for insertion
        records = [tuple(x) for x in df.to_numpy()]

        # The INSERT query with ON DUPLICATE KEY UPDATE to handle idempotency
        insert_query = f"""
            INSERT INTO {table_name} (
                transaction_id, transaction_date, transaction_time, transaction_qty,
                store_id, store_location, product_id, unit_price,
                product_category, product_type, product_detail
            ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
            ON DUPLICATE KEY UPDATE
                transaction_date=VALUES(transaction_date),
                transaction_time=VALUES(transaction_time),
                transaction_qty=VALUES(transaction_qty),
                store_id=VALUES(store_id),
                store_location=VALUES(store_location),
                product_id=VALUES(product_id),
                unit_price=VALUES(unit_price),
                product_category=VALUES(product_category),
                product_type=VALUES(product_type),
                product_detail=VALUES(product_detail)
        """

        cursor.executemany(insert_query, records)
        conn.commit()
        print(f"✅ Load complete! Inserted/updated {cursor.rowcount} records.")

    except mysql.connector.Error as err:
        logging.error(f"❌ MySQL Error: {err}")
        if 'conn' in locals() and conn.is_connected():
            conn.rollback()
    finally:
        if 'cursor' in locals() and cursor is not None:
            cursor.close()
        if 'conn' in locals() and conn.is_connected():
            conn.close()
        print("MySQL connection closed.")

if __name__ == "__main__":
    input_file = "Coffee Shop Sales2.csv"
    table_name = "sales_data"
    
    db_config = {
        'host': 'localhost',
        'user': 'root',
        'password': 'Whatsnew2711',
        'database': 'coffee_shop_sales'
    }

    start_date = input("Enter start date (YYYY-MM-DD): ").strip()
    end_date = input("Enter end date (YYYY-MM-DD): ").strip()

    df_raw = extract_data(input_file)
    if df_raw is not None:
        df_transformed = transform_data(df_raw, start_date, end_date)
        if df_transformed is not None:
            load_data_to_mysql(df_transformed, db_config, table_name)


Enter start date (YYYY-MM-DD):  2025-01-01
Enter end date (YYYY-MM-DD):  2025-01-05


📥 Extracting data from 'Coffee Shop Sales2.csv'...
🔄 Transforming data...
🚀 Loading data to MySQL...
Successfully connected to MySQL database.
✅ Load complete! Inserted/updated 2742 records.
MySQL connection closed.


In [20]:
import pandas as pd
import mysql.connector
import psycopg2
import logging
from datetime import datetime, timedelta

# Set up logging for visibility and easy debugging.
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# =========================================================
# 1️⃣ Database Connection Configurations
# =========================================================
# Configuration for the MySQL OLTP database.
MYSQL_HOST = "localhost"
MYSQL_PORT = 3306
MYSQL_USER = "root"
MYSQL_PASSWORD = "Whatsnew2711"
MYSQL_DB = 'coffee_shop_sales'

# Configuration for the PostgreSQL OLAP database.
PG_HOST = "localhost"
PG_PORT = 5432
PG_USER = "postgres"
PG_PASSWORD = "Whatsnew2711"
PG_DB = "coffee_sales_data"

def get_postgres_connection():
    """Establishes and returns a PostgreSQL database connection."""
    try:
        conn = psycopg2.connect(
            host=PG_HOST,
            port=PG_PORT,
            user=PG_USER,
            password=PG_PASSWORD,
            dbname=PG_DB
        )
        return conn
    except psycopg2.Error as e:
        logging.error(f"❌ Failed to connect to PostgreSQL: {e}")
        return None

# =========================================================
# 2️⃣ ETL Functions
# =========================================================

def extract_data_from_mysql(date_to_process):
    """
    Extracts daily transaction data from MySQL OLTP database
    for a specific date.
    """
    logging.info(f"📥 Extracting data from MySQL for {date_to_process}...")
    conn = None
    cursor = None
    try:
        conn = mysql.connector.connect(
            host=MYSQL_HOST,
            port=MYSQL_PORT,
            user=MYSQL_USER,
            password=MYSQL_PASSWORD,
            database=MYSQL_DB
        )
        cursor = conn.cursor()
        query = f"""
        SELECT
            transaction_id,
            transaction_date,
            transaction_time,
            transaction_qty,
            store_id,
            store_location,
            product_id,
            unit_price,
            product_category,
            product_type,
            product_detail
        FROM sales_data
        WHERE transaction_date = '{date_to_process}';
        """
        df_raw = pd.read_sql(query, conn)
        logging.info(f"✅ Extracted {len(df_raw)} records.")
        return df_raw
    except mysql.connector.Error as err:
        logging.error(f"❌ MySQL Error: {err}")
        if 'conn' in locals() and conn.is_connected():
            conn.rollback()
        return None
    finally:
        if 'cursor' in locals() and cursor is not None:
            cursor.close()
        if 'conn' in locals() and conn.is_connected():
            conn.close()
            logging.info("MySQL connection closed.")

def transform_data(df_raw):
    """
    Transforms raw MySQL data into star schema DataFrames that match PostgreSQL tables.
    Returns: dim_date, dim_time, dim_store, dim_product, df_fact
    """
    logging.info("🔄 Transforming data for OLAP...")

    # ----------------------------
    # Convert transaction_date to datetime
    # ----------------------------
    if "transaction_date" in df_raw.columns:
        df_raw["transaction_date"] = pd.to_datetime(df_raw["transaction_date"], errors="coerce")

    # ----------------------------
    # Convert transaction_time from timedelta to time
    # ----------------------------
    if "transaction_time" in df_raw.columns:
        if pd.api.types.is_timedelta64_dtype(df_raw["transaction_time"]):
            df_raw["transaction_time"] = df_raw["transaction_time"].apply(
                lambda x: (pd.Timestamp("00:00:00") + x).time() if pd.notnull(x) else None
            )
        else:
            df_raw["transaction_time"] = pd.to_datetime(
                df_raw["transaction_time"].astype(str), errors="coerce"
            ).dt.time

    # Drop rows where conversion failed
    df_raw = df_raw.dropna(subset=["transaction_date", "transaction_time"])

    # ----------------------------
    # Calculate total_amount
    # ----------------------------
    df_raw["total_amount"] = df_raw["transaction_qty"] * df_raw["unit_price"]

    # ----------------------------
    # Dim Date
    # ----------------------------
    dim_date = df_raw[["transaction_date"]].drop_duplicates().reset_index(drop=True)
    dim_date["day"] = dim_date["transaction_date"].dt.day
    dim_date["month"] = dim_date["transaction_date"].dt.month
    dim_date["year"] = dim_date["transaction_date"].dt.year
    dim_date["weekday"] = dim_date["transaction_date"].dt.day_name()

    # ----------------------------
    # Dim Time
    # ----------------------------
    dim_time = df_raw[["transaction_time"]].drop_duplicates().reset_index(drop=True)
    dim_time["hour"] = dim_time["transaction_time"].apply(lambda x: x.hour)
    dim_time["minute"] = dim_time["transaction_time"].apply(lambda x: x.minute)
    dim_time["second"] = dim_time["transaction_time"].apply(lambda x: x.second)

    # ----------------------------
    # Dim Store
    # ----------------------------
    dim_store = df_raw[["store_id", "store_location"]].drop_duplicates().reset_index(drop=True)

    # ----------------------------
    # Dim Product
    # ----------------------------
    dim_product = df_raw[["product_id", "product_category", "product_type", "product_detail", "unit_price"]].drop_duplicates().reset_index(drop=True)

    # ----------------------------
    # Fact Table
    # ----------------------------
    df_fact = df_raw.copy()
    df_fact = df_fact.merge(dim_date, on="transaction_date")
    df_fact = df_fact.merge(dim_time, on="transaction_time")
    df_fact = df_fact.merge(dim_store, on=["store_id", "store_location"])
    df_fact = df_fact.merge(dim_product, on=["product_id", "product_category", "product_type", "product_detail", "unit_price"])

    df_fact = df_fact[[
        "transaction_id",
        "transaction_date",  # maps to dim_date.date_id in load
        "transaction_time",  # maps to dim_time.time_id
        "store_id",          # maps to dim_store.store_id
        "product_id",        # maps to dim_product.product_id
        "transaction_qty",
        "total_amount"
    ]]

    logging.info(f"✅ Transformation completed. Records: {len(df_fact)}")
    return dim_date, dim_time, dim_store, dim_product, df_fact

def load_data_to_postgres(dim_date, dim_time, dim_store, dim_product, df_fact):
    """
    Loads transformed data into PostgreSQL OLAP database
    by checking for existence before inserting to avoid duplicates.
    """
    logging.info("📤 Loading data to PostgreSQL...")
    conn = None
    try:
        conn = get_postgres_connection()
        if not conn:
            return
        
        cur = conn.cursor()

        # -----------------------------
        # Dim Date (Check then Insert)
        # -----------------------------
        date_id_map = {}
        for _, row in dim_date.iterrows():
            cur.execute("SELECT date_id FROM dim_date WHERE transaction_date = %s;", (row["transaction_date"].date(),))
            result = cur.fetchone()

            if result:
                date_id_map[row["transaction_date"]] = result[0]
            else:
                cur.execute("""
                    INSERT INTO dim_date (transaction_date, day, month, year, weekday)
                    VALUES (%s, %s, %s, %s, %s)
                    RETURNING date_id;
                """, (
                    row["transaction_date"].date(),
                    int(row["day"]),
                    int(row["month"]),
                    int(row["year"]),
                    row["weekday"]
                ))
                date_id_map[row["transaction_date"]] = cur.fetchone()[0]

        # -----------------------------
        # Dim Time (Check then Insert)
        # -----------------------------
        time_id_map = {}
        for _, row in dim_time.iterrows():
            cur.execute("SELECT time_id FROM dim_time WHERE transaction_time = %s;", (row["transaction_time"],))
            result = cur.fetchone()

            if result:
                time_id_map[row["transaction_time"]] = result[0]
            else:
                cur.execute("""
                    INSERT INTO dim_time (transaction_time, hour, minute, second)
                    VALUES (%s, %s, %s, %s)
                    RETURNING time_id;
                """, (
                    row["transaction_time"],
                    int(row["hour"]),
                    int(row["minute"]),
                    int(row["second"])
                ))
                time_id_map[row["transaction_time"]] = cur.fetchone()[0]

        # -----------------------------
        # Dim Store (Check then Insert)
        # -----------------------------
        store_id_map = {}
        for _, row in dim_store.iterrows():
            cur.execute("SELECT store_id FROM dim_store WHERE store_location = %s;", (row["store_location"],))
            result = cur.fetchone()

            if result:
                store_id_map[row["store_id"]] = result[0]
            else:
                cur.execute("""
                    INSERT INTO dim_store (store_location)
                    VALUES (%s)
                    RETURNING store_id;
                """, (row["store_location"],))
                store_id_map[row["store_id"]] = cur.fetchone()[0]

        # -----------------------------
        # Dim Product (Check then Insert)
        # -----------------------------
        product_id_map = {}
        for _, row in dim_product.iterrows():
            cur.execute("""
                SELECT product_id FROM dim_product
                WHERE product_category = %s AND product_type = %s AND product_detail = %s;
            """, (row["product_category"], row["product_type"], row["product_detail"]))
            result = cur.fetchone()

            if result:
                product_id_map[row["product_id"]] = result[0]
            else:
                cur.execute("""
                    INSERT INTO dim_product (product_category, product_type, product_detail, unit_price)
                    VALUES (%s, %s, %s, %s)
                    RETURNING product_id;
                """, (
                    row["product_category"],
                    row["product_type"],
                    row["product_detail"],
                    float(row["unit_price"])
                ))
                product_id_map[row["product_id"]] = cur.fetchone()[0]
        
        conn.commit()

        # -----------------------------
        # Fact Table (Check then Insert)
        # -----------------------------
        for _, row in df_fact.iterrows():
            cur.execute("SELECT transaction_id FROM fact_sales WHERE transaction_id = %s;", (row["transaction_id"],))
            result = cur.fetchone()
            
            if not result:
                cur.execute("""
                    INSERT INTO fact_sales (transaction_id, date_id, time_id, store_id, product_id, transaction_qty, total_amount)
                    VALUES (%s, %s, %s, %s, %s, %s, %s);
                """, (
                    row["transaction_id"],
                    date_id_map[row["transaction_date"]],
                    time_id_map[row["transaction_time"]],
                    store_id_map[row["store_id"]],
                    product_id_map[row["product_id"]],
                    int(row["transaction_qty"]),
                    float(row["total_amount"])
                ))
        
        conn.commit()
        cur.close()
        logging.info("✅ Data loaded to PostgreSQL successfully!")

    except Exception as e:
        logging.error(f"❌ Failed to load data to PostgreSQL: {e}")
        if conn:
            conn.rollback()
    finally:
        if conn:
            conn.close()

# =========================================================
# 3️⃣ Main Execution Logic for Backdating
# =========================================================

def run_backdate(start_date_str, end_date_str):
    """
    Runs the full ETL pipeline for a specified date range.
    """
    try:
        # Convert string dates to datetime.date objects for easier iteration
        start_date = datetime.strptime(start_date_str, '%Y-%m-%d').date()
        end_date = datetime.strptime(end_date_str, '%Y-%m-%d').date()
    except ValueError:
        logging.error("❌ Invalid date format. Please use YYYY-MM-DD.")
        return

    logging.info(f"⏳ Starting backdating process from {start_date} to {end_date}...")

    current_date = start_date
    while current_date <= end_date:
        date_to_process = current_date.strftime('%Y-%m-%d')
        logging.info(f"📆 Processing date: {date_to_process}")

        # --- Extract ---
        df_raw = extract_data_from_mysql(date_to_process)
        
        # Check if extraction was successful and data exists
        if df_raw is not None and not df_raw.empty:
            # --- Transform ---
            try:
                dim_date, dim_time, dim_store, dim_product, df_fact = transform_data(df_raw)
                # --- Load ---
                load_data_to_postgres(dim_date, dim_time, dim_store, dim_product, df_fact)
                logging.info(f"✅ Completed ETL for: {date_to_process}")
            except Exception as e:
                logging.error(f"❌ ETL process failed for {date_to_process}: {e}")
        else:
            logging.warning(f"⚠️ No data found for {date_to_process} or extraction failed. Skipping.")
        
        # Increment to the next day
        current_date += timedelta(days=1)
    
    logging.info("🎉 Backdating process finished.")

if __name__ == "__main__":
    start_input = input("Enter start date (YYYY-MM-DD): ").strip()
    end_input = input("Enter end date (YYYY-MM-DD): ").strip()
    run_backdate(start_input, end_input)

Enter start date (YYYY-MM-DD):  2025-01-01
Enter end date (YYYY-MM-DD):  2025-01-01


INFO:root:⏳ Starting backdating process from 2025-01-01 to 2025-01-01...
INFO:root:📆 Processing date: 2025-01-01
INFO:root:📥 Extracting data from MySQL for 2025-01-01...
C:\Users\khang\AppData\Local\Temp\ipykernel_13876\2017162868.py:79: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_raw = pd.read_sql(query, conn)
INFO:root:✅ Extracted 549 records.
INFO:root:MySQL connection closed.
INFO:root:🔄 Transforming data for OLAP...
INFO:root:✅ Transformation completed. Records: 549
INFO:root:📤 Loading data to PostgreSQL...
INFO:root:✅ Data loaded to PostgreSQL successfully!
INFO:root:✅ Completed ETL for: 2025-01-01
INFO:root:🎉 Backdating process finished.


In [10]:
import pandas as pd
import mysql.connector
import psycopg2
import logging
from datetime import datetime, timedelta

# Set up logging for visibility and easy debugging.
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# =========================================================
# 1️⃣ Database Connection Configurations
# =========================================================
# Configuration for the MySQL OLTP database.
MYSQL_HOST = "localhost"
MYSQL_PORT = 3306
MYSQL_USER = "root"
MYSQL_PASSWORD = "Whatsnew2711"
MYSQL_DB = 'coffee_shop_sales'

# Configuration for the PostgreSQL OLAP database.
PG_HOST = "localhost"
PG_PORT = 5432
PG_USER = "postgres"
PG_PASSWORD = "Whatsnew2711"
PG_DB = "coffee_sales_data"

def get_postgres_connection():
    """Establishes and returns a PostgreSQL database connection."""
    try:
        conn = psycopg2.connect(
            host=PG_HOST,
            port=PG_PORT,
            user=PG_USER,
            password=PG_PASSWORD,
            dbname=PG_DB
        )
        return conn
    except psycopg2.Error as e:
        logging.error(f"❌ Failed to connect to PostgreSQL: {e}")
        return None

# =========================================================
# 2️⃣ ETL Functions
# =========================================================

def extract_data_from_mysql(date_to_process):
    """
    Extracts daily transaction data from MySQL OLTP database
    for a specific date.
    """
    logging.info(f"📥 Extracting data from MySQL for {date_to_process}...")
    conn = None
    cursor = None
    try:
        conn = mysql.connector.connect(
            host=MYSQL_HOST,
            port=MYSQL_PORT,
            user=MYSQL_USER,
            password=MYSQL_PASSWORD,
            database=MYSQL_DB
        )
        cursor = conn.cursor()
        query = f"""
        SELECT
            transaction_id,
            transaction_date,
            transaction_time,
            transaction_qty,
            store_id,
            store_location,
            product_id,
            unit_price,
            product_category,
            product_type,
            product_detail
        FROM sales_data
        WHERE transaction_date = '{date_to_process}';
        """
        df_raw = pd.read_sql(query, conn)
        logging.info(f"✅ Extracted {len(df_raw)} records.")
        return df_raw
    except mysql.connector.Error as err:
        logging.error(f"❌ MySQL Error: {err}")
        if 'conn' in locals() and conn.is_connected():
            conn.rollback()
        return None
    finally:
        if 'cursor' in locals() and cursor is not None:
            cursor.close()
        if 'conn' in locals() and conn.is_connected():
            conn.close()
            logging.info("MySQL connection closed.")

def transform_data(df_raw):
    """
    Transforms raw MySQL data into star schema DataFrames that match PostgreSQL tables.
    Returns: dim_date, dim_time, dim_store, dim_product, df_fact
    """
    logging.info("🔄 Transforming data for OLAP...")

    # ----------------------------
    # Convert transaction_date to datetime
    # ----------------------------
    if "transaction_date" in df_raw.columns:
        df_raw["transaction_date"] = pd.to_datetime(df_raw["transaction_date"], errors="coerce")

    # ----------------------------
    # Convert transaction_time from timedelta to time
    # ----------------------------
    if "transaction_time" in df_raw.columns:
        if pd.api.types.is_timedelta64_dtype(df_raw["transaction_time"]):
            df_raw["transaction_time"] = df_raw["transaction_time"].apply(
                lambda x: (pd.Timestamp("00:00:00") + x).time() if pd.notnull(x) else None
            )
        else:
            df_raw["transaction_time"] = pd.to_datetime(
                df_raw["transaction_time"].astype(str), errors="coerce"
            ).dt.time

    # Drop rows where conversion failed
    df_raw = df_raw.dropna(subset=["transaction_date", "transaction_time"])

    # ----------------------------
    # Calculate total_amount
    # ----------------------------
    df_raw["total_amount"] = df_raw["transaction_qty"] * df_raw["unit_price"]

    # ----------------------------
    # Dim Date
    # ----------------------------
    dim_date = df_raw[["transaction_date"]].drop_duplicates().reset_index(drop=True)
    dim_date["day"] = dim_date["transaction_date"].dt.day
    dim_date["month"] = dim_date["transaction_date"].dt.month
    dim_date["year"] = dim_date["transaction_date"].dt.year
    dim_date["weekday"] = dim_date["transaction_date"].dt.day_name()

    # ----------------------------
    # Dim Time
    # ----------------------------
    dim_time = df_raw[["transaction_time"]].drop_duplicates().reset_index(drop=True)
    dim_time["hour"] = dim_time["transaction_time"].apply(lambda x: x.hour)
    dim_time["minute"] = dim_time["transaction_time"].apply(lambda x: x.minute)
    dim_time["second"] = dim_time["transaction_time"].apply(lambda x: x.second)

    # ----------------------------
    # Dim Store
    # ----------------------------
    dim_store = df_raw[["store_id", "store_location"]].drop_duplicates().reset_index(drop=True)

    # ----------------------------
    # Dim Product
    # ----------------------------
    dim_product = df_raw[["product_id", "product_category", "product_type", "product_detail", "unit_price"]].drop_duplicates().reset_index(drop=True)

    # ----------------------------
    # Fact Table
    # ----------------------------
    df_fact = df_raw.copy()
    df_fact = df_fact.merge(dim_date, on="transaction_date")
    df_fact = df_fact.merge(dim_time, on="transaction_time")
    df_fact = df_fact.merge(dim_store, on=["store_id", "store_location"])
    df_fact = df_fact.merge(dim_product, on=["product_id", "product_category", "product_type", "product_detail", "unit_price"])

    df_fact = df_fact[[
        "transaction_id",
        "transaction_date",  # maps to dim_date.date_id in load
        "transaction_time",  # maps to dim_time.time_id
        "store_id",          # maps to dim_store.store_id
        "product_id",        # maps to dim_product.product_id
        "transaction_qty",
        "total_amount"
    ]]

    logging.info(f"✅ Transformation completed. Records: {len(df_fact)}")
    return dim_date, dim_time, dim_store, dim_product, df_fact

def load_data_to_postgres(dim_date, dim_time, dim_store, dim_product, df_fact):
    """
    Loads transformed data into PostgreSQL OLAP database
    by checking for existence before inserting to avoid duplicates.
    """
    logging.info("📤 Loading data to PostgreSQL...")
    conn = None
    try:
        conn = get_postgres_connection()
        if not conn:
            return
        
        cur = conn.cursor()

        # -----------------------------
        # Dim Date (Check then Insert)
        # -----------------------------
        date_id_map = {}
        for _, row in dim_date.iterrows():
            cur.execute("SELECT date_id FROM dim_date WHERE transaction_date = %s;", (row["transaction_date"].date(),))
            result = cur.fetchone()

            if result:
                date_id_map[row["transaction_date"]] = result[0]
            else:
                cur.execute("""
                    INSERT INTO dim_date (transaction_date, day, month, year, weekday)
                    VALUES (%s, %s, %s, %s, %s)
                    RETURNING date_id;
                """, (
                    row["transaction_date"].date(),
                    int(row["day"]),
                    int(row["month"]),
                    int(row["year"]),
                    row["weekday"]
                ))
                date_id_map[row["transaction_date"]] = cur.fetchone()[0]

        # -----------------------------
        # Dim Time (Check then Insert)
        # -----------------------------
        time_id_map = {}
        for _, row in dim_time.iterrows():
            cur.execute("SELECT time_id FROM dim_time WHERE transaction_time = %s;", (row["transaction_time"],))
            result = cur.fetchone()

            if result:
                time_id_map[row["transaction_time"]] = result[0]
            else:
                cur.execute("""
                    INSERT INTO dim_time (transaction_time, hour, minute, second)
                    VALUES (%s, %s, %s, %s)
                    RETURNING time_id;
                """, (
                    row["transaction_time"],
                    int(row["hour"]),
                    int(row["minute"]),
                    int(row["second"])
                ))
                time_id_map[row["transaction_time"]] = cur.fetchone()[0]

        # -----------------------------
        # Dim Store (Check then Insert)
        # -----------------------------
        store_id_map = {}
        for _, row in dim_store.iterrows():
            cur.execute("SELECT store_id FROM dim_store WHERE store_location = %s;", (row["store_location"],))
            result = cur.fetchone()

            if result:
                store_id_map[row["store_id"]] = result[0]
            else:
                cur.execute("""
                    INSERT INTO dim_store (store_location)
                    VALUES (%s)
                    RETURNING store_id;
                """, (row["store_location"],))
                store_id_map[row["store_id"]] = cur.fetchone()[0]

        # -----------------------------
        # Dim Product (Check then Insert)
        # -----------------------------
        product_id_map = {}
        for _, row in dim_product.iterrows():
            cur.execute("""
                SELECT product_id FROM dim_product
                WHERE product_category = %s AND product_type = %s AND product_detail = %s;
            """, (row["product_category"], row["product_type"], row["product_detail"]))
            result = cur.fetchone()

            if result:
                product_id_map[row["product_id"]] = result[0]
            else:
                cur.execute("""
                    INSERT INTO dim_product (product_category, product_type, product_detail, unit_price)
                    VALUES (%s, %s, %s, %s)
                    RETURNING product_id;
                """, (
                    row["product_category"],
                    row["product_type"],
                    row["product_detail"],
                    float(row["unit_price"])
                ))
                product_id_map[row["product_id"]] = cur.fetchone()[0]
        
        conn.commit()

        # -----------------------------
        # Fact Table (Check then Insert)
        # -----------------------------
        for _, row in df_fact.iterrows():
            cur.execute("SELECT transaction_id FROM fact_sales WHERE transaction_id = %s;", (row["transaction_id"],))
            result = cur.fetchone()
            
            if not result:
                cur.execute("""
                    INSERT INTO fact_sales (transaction_id, date_id, time_id, store_id, product_id, transaction_qty, total_amount)
                    VALUES (%s, %s, %s, %s, %s, %s, %s);
                """, (
                    row["transaction_id"],
                    date_id_map[row["transaction_date"]],
                    time_id_map[row["transaction_time"]],
                    store_id_map[row["store_id"]],
                    product_id_map[row["product_id"]],
                    int(row["transaction_qty"]),
                    float(row["total_amount"])
                ))
        
        conn.commit()
        cur.close()
        logging.info("✅ Data loaded to PostgreSQL successfully!")

    except Exception as e:
        logging.error(f"❌ Failed to load data to PostgreSQL: {e}")
        if conn:
            conn.rollback()
    finally:
        if conn:
            conn.close()

# =========================================================
# 3️⃣ Main Execution Logic for Backdating
# =========================================================

def get_latest_date_from_postgres():
    """
    Queries PostgreSQL to find the most recent transaction date in the
    fact_sales table.
    """
    logging.info("🔍 Checking for the latest date in PostgreSQL...")
    conn = None
    try:
        conn = get_postgres_connection()
        if not conn:
            return None
        
        cur = conn.cursor()
        # Join fact_sales with dim_date to get the actual date
        cur.execute("""
            SELECT MAX(T2.transaction_date)
            FROM fact_sales AS T1
            JOIN dim_date AS T2
            ON T1.date_id = T2.date_id;
        """)
        latest_date = cur.fetchone()[0]
        cur.close()

        if latest_date:
            logging.info(f"✅ Latest date found in database: {latest_date}")
            return latest_date
        else:
            logging.warning("⚠️ No data found in fact_sales table. Starting from default date.")
            return None
    except Exception as e:
        logging.error(f"❌ Failed to retrieve latest date from PostgreSQL: {e}")
        return None
    finally:
        if conn:
            conn.close()

def run_daily_etl():
    """
    Runs the ETL pipeline for a single day, determined by the latest
    data in the PostgreSQL database.
    """
    # Find the latest date in the OLAP database
    latest_date_in_db = get_latest_date_from_postgres()
    
    # Determine the date to process. If no data exists, start from the beginning.
    if latest_date_in_db:
        # Load the next day's data
        date_to_process = (latest_date_in_db + timedelta(days=1)).strftime('%Y-%m-%d')
    else:
        # Default start date for the backfilling.
        date_to_process = '2025-01-01'

    logging.info(f"📆 Processing date: {date_to_process}")

    # --- Extract ---
    df_raw = extract_data_from_mysql(date_to_process)
    
    # Check if extraction was successful and data exists
    if df_raw is not None and not df_raw.empty:
        # --- Transform ---
        try:
            dim_date, dim_time, dim_store, dim_product, df_fact = transform_data(df_raw)
            # --- Load ---
            load_data_to_postgres(dim_date, dim_time, dim_store, dim_product, df_fact)
            logging.info(f"✅ Completed ETL for: {date_to_process}")
        except Exception as e:
            logging.error(f"❌ ETL process failed for {date_to_process}: {e}")
    else:
        logging.warning(f"⚠️ No data found for {date_to_process} or extraction failed. Skipping.")

if __name__ == "__main__":
    run_daily_etl()



2025-08-20 19:38:35,596 - INFO - 🔍 Checking for the latest date in PostgreSQL...
2025-08-20 19:38:35,739 - INFO - ✅ Latest date found in database: 2025-01-02
2025-08-20 19:38:35,743 - INFO - 📆 Processing date: 2025-01-03
2025-08-20 19:38:35,744 - INFO - 📥 Extracting data from MySQL for 2025-01-03...
C:\Users\khang\AppData\Local\Temp\ipykernel_2500\880047466.py:79: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_raw = pd.read_sql(query, conn)
2025-08-20 19:38:36,513 - INFO - ✅ Extracted 582 records.
2025-08-20 19:38:36,520 - INFO - MySQL connection closed.
2025-08-20 19:38:36,523 - INFO - 🔄 Transforming data for OLAP...
2025-08-20 19:38:36,703 - INFO - ✅ Transformation completed. Records: 582
2025-08-20 19:38:36,707 - INFO - 📤 Loading data to PostgreSQL...
2025-08-20 19:38:38,259 - INFO - ✅ Data loaded to PostgreSQL successfully!
2

In [4]:
import os
os.getenv("AIRFLOW_HOME")

In [16]:
import os
from datetime import datetime, timedelta
import pandas as pd
from etl_oltp import extract_data, transform_data, load_data

INPUT_FILE = "Coffee Shop Sales2.csv"
OUTPUT_FILE = "transformed.csv"

def get_next_date_to_process(output_csv, df_raw):
    """
    Determine the next date to backdate based on the last date in the CSV.
    If the CSV doesn't exist, return the earliest date in the raw data.
    """
    df_raw['transaction_date'] = pd.to_datetime(df_raw['transaction_date'], errors='coerce')
    
    if not os.path.exists(output_csv):
        return df_raw['transaction_date'].min()

    df_existing = pd.read_csv(output_csv)
    df_existing['transaction_date'] = pd.to_datetime(df_existing['transaction_date'], errors='coerce')
    df_existing = df_existing.dropna(subset=['transaction_date'])
    last_date = df_existing['transaction_date'].max()

    return last_date + timedelta(days=1)


def run_daily_incremental():
    """
    Run ETL for the next unprocessed date in the raw CSV.
    """
    df_raw = extract_data(INPUT_FILE)
    if df_raw is None:
        print("❌ Extraction failed.")
        return

    next_date = get_next_date_to_process(OUTPUT_FILE, df_raw)

    df_raw['transaction_date'] = pd.to_datetime(df_raw['transaction_date'], errors='coerce')
    df_raw = df_raw.dropna(subset=['transaction_date'])

    last_raw_date = df_raw['transaction_date'].max()
    if next_date > last_raw_date:
        print("✅ No new data to process. All dates up to latest available are loaded.")
        return

    start_date_str = next_date.strftime('%Y-%m-%d')
    end_date_str = next_date.strftime('%Y-%m-%d')

    print(f"📆 Processing daily backdate for: {start_date_str}")

    # --- Transform and load ---
    df_transformed = transform_data(df_raw, start_date_str, end_date_str)
    if df_transformed.empty:
        print(f"⚠ No data found for {start_date_str}")
        return

    load_data(df_transformed, OUTPUT_FILE, start_date_str, end_date_str)
    print(f"✅ Daily backdate for {start_date_str} completed.")


In [19]:
import pandas as pd
import os
import mysql.connector
import logging
from datetime import datetime, timedelta

# Set up logging for better visibility
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# =========================================================
# 1️⃣ Database Connection Configurations
# =========================================================
# Configuration for the MySQL OLTP database.
MYSQL_HOST = "localhost"
MYSQL_PORT = 3306
MYSQL_USER = "root"
MYSQL_PASSWORD = "Whatsnew2711"
MYSQL_DB = 'coffee_shop_sales'

def get_mysql_connection():
    """Establishes and returns a MySQL database connection."""
    try:
        conn = mysql.connector.connect(
            host=MYSQL_HOST,
            port=MYSQL_PORT,
            user=MYSQL_USER,
            password=MYSQL_PASSWORD,
            database=MYSQL_DB
        )
        return conn
    except mysql.connector.Error as err:
        logging.error(f"❌ Failed to connect to MySQL: {err}")
        return None

# =========================================================
# 2️⃣ ETL Functions
# =========================================================

def extract_data(input_csv):
    """
    Extracts data from a specified CSV file into a pandas DataFrame.
    """
    logging.info(f"📥 Extracting data from '{input_csv}'...")
    try:
        df = pd.read_csv(input_csv, sep=";")
        return df
    except FileNotFoundError:
        logging.error(f"❌ Error: The file '{input_csv}' was not found.")
        return None

def transform_data(df, start_offset_date, end_offset_date):
    """
    Transforms the raw DataFrame by:
    - Filtering by date range
    - Formatting dates & times
    - Mapping store locations
    - Converting currency
    - Regenerating unique transaction IDs
    """
    logging.info("🔄 Transforming data...")

    # Convert date and time
    df['transaction_date'] = pd.to_datetime(df['transaction_date'], errors='coerce')
    df['transaction_time'] = pd.to_datetime(
        df['transaction_time'], format='%H:%M:%S', errors='coerce'
    ).dt.strftime('%H:%M:%S')

    # Filter by date range
    start_date = pd.to_datetime(start_offset_date)
    end_date = pd.to_datetime(end_offset_date)
    df_filtered = df[
        (df['transaction_date'] >= start_date) & (df['transaction_date'] <= end_date)
    ].copy()

    if df_filtered.empty:
        logging.warning(f"⚠ No data found for the range {start_offset_date} to {end_offset_date}.")
        return df_filtered

    # Format date for output
    df_filtered['transaction_date'] = df_filtered['transaction_date'].dt.strftime('%Y-%m-%d')

    # Map store locations
    location_map = {
        "Hell's Kitchen": "East Campus",
        "Astoria": "West Campus",
        "Lower Manhattan": "Main Campus"
    }
    df_filtered['store_location'] = df_filtered['store_location'].replace(location_map)

    # Convert unit_price
    df_filtered['unit_price'] = (df_filtered['unit_price'] * 15).round(2)

    # Convert key columns to string safely
    cols_to_convert = ['transaction_id', 'store_id', 'product_id']
    df_filtered[cols_to_convert] = df_filtered[cols_to_convert].apply(
        lambda col: col.dropna().astype(int).astype(str)
    )

    # Force unique transaction IDs by appending the date
    df_filtered['transaction_id'] = (
        df_filtered['transaction_id'].astype(str) + "_" +
        pd.to_datetime(df_filtered['transaction_date']).dt.strftime('%Y%m%d')
    )
    
    # Reorder columns to match the MySQL table structure
    df_transformed = df_filtered.reindex(columns=[
        'transaction_id', 'transaction_date', 'transaction_time', 'transaction_qty',
        'store_id', 'store_location', 'product_id', 'unit_price',
        'product_category', 'product_type', 'product_detail'
    ], fill_value=None)

    return df_transformed


def load_data_to_mysql(df, table_name):
    """
    Loads transformed data into a MySQL table using the ON DUPLICATE KEY UPDATE.
    """
    if df.empty:
        logging.warning("⚠ No data to load to MySQL.")
        return

    logging.info("🚀 Loading data to MySQL...")
    conn = None
    cursor = None
    try:
        conn = get_mysql_connection()
        if not conn:
            return
        
        cursor = conn.cursor()
        logging.info("Successfully connected to MySQL database.")

        # Prepare the list of tuples for insertion
        records = [tuple(x) for x in df.to_numpy()]

        # The INSERT query with ON DUPLICATE KEY UPDATE to handle idempotency
        insert_query = f"""
            INSERT INTO {table_name} (
                transaction_id, transaction_date, transaction_time, transaction_qty,
                store_id, store_location, product_id, unit_price,
                product_category, product_type, product_detail
            ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
            ON DUPLICATE KEY UPDATE
                transaction_date=VALUES(transaction_date),
                transaction_time=VALUES(transaction_time),
                transaction_qty=VALUES(transaction_qty),
                store_id=VALUES(store_id),
                store_location=VALUES(store_location),
                product_id=VALUES(product_id),
                unit_price=VALUES(unit_price),
                product_category=VALUES(product_category),
                product_type=VALUES(product_type),
                product_detail=VALUES(product_detail)
        """

        cursor.executemany(insert_query, records)
        conn.commit()
        logging.info(f"✅ Load complete! Inserted/updated {cursor.rowcount} records.")

    except mysql.connector.Error as err:
        logging.error(f"❌ MySQL Error: {err}")
        if conn and conn.is_connected():
            conn.rollback()
    finally:
        if cursor is not None:
            cursor.close()
        if conn and conn.is_connected():
            conn.close()
            logging.info("MySQL connection closed.")

def get_latest_date_from_mysql():
    """
    Queries MySQL to find the most recent transaction date in the
    sales_data table.
    """
    logging.info("🔍 Checking for the latest date in MySQL...")
    conn = None
    cursor = None
    try:
        conn = get_mysql_connection()
        if not conn:
            return None
        
        cursor = conn.cursor()
        cursor.execute("""
            SELECT MAX(transaction_date)
            FROM sales_data;
        """)
        latest_date = cursor.fetchone()[0]
        
        if latest_date:
            logging.info(f"✅ Latest date found in database: {latest_date}")
            return latest_date
        else:
            logging.warning("⚠️ No data found in sales_data table. Starting from default date.")
            return None
    except mysql.connector.Error as err:
        logging.error(f"❌ Failed to retrieve latest date from MySQL: {err}")
        return None
    finally:
        if cursor:
            cursor.close()
        if conn:
            conn.close()

# =========================================================
# 3️⃣ Main Execution Logic for Daily Schedule
# =========================================================
if __name__ == "__main__":
    input_file = "Coffee Shop Sales2.csv"
    table_name = "sales_data"
    
    # Get the latest date from the database
    latest_date_in_db = get_latest_date_from_mysql()
    
    # Determine the date to process. If no data exists, start from the beginning.
    if latest_date_in_db:
        # Load the next day's data
        date_to_process = (latest_date_in_db + timedelta(days=1)).strftime('%Y-%m-%d')
    else:
        # Default start date for the backfilling.
        date_to_process = '2025-01-01'

    logging.info(f"📆 Processing date: {date_to_process}")

    df_raw = extract_data(input_file)
    if df_raw is not None:
        # The start and end date for transformation is the same, as we process one day at a time
        df_transformed = transform_data(df_raw, date_to_process, date_to_process)
        if df_transformed is not None:
            load_data_to_mysql(df_transformed, table_name)


2025-08-20 19:53:47,884 - INFO - 🔍 Checking for the latest date in MySQL...
2025-08-20 19:53:47,946 - INFO - ✅ Latest date found in database: 2025-01-05
2025-08-20 19:53:47,949 - INFO - 📆 Processing date: 2025-01-06
2025-08-20 19:53:47,951 - INFO - 📥 Extracting data from 'Coffee Shop Sales2.csv'...
2025-08-20 19:53:48,222 - INFO - 🔄 Transforming data...
2025-08-20 19:53:51,288 - INFO - 🚀 Loading data to MySQL...
2025-08-20 19:53:51,358 - INFO - Successfully connected to MySQL database.
2025-08-20 19:53:51,510 - INFO - ✅ Load complete! Inserted/updated 509 records.
2025-08-20 19:53:51,519 - INFO - MySQL connection closed.
